In [ ]:
import os
import pathlib
import ast

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models
    
!conda install -y cudnn 

import tensorflow as tf
tf.test.gpu_device_name()
tf.test.is_built_with_cuda()

In [ ]:
!ln -s /usr/local/cuda-11.0/targets/x86_64-linux/lib/libcusolver.so.10 /usr/local/cuda-11.0/targets/x86_64-linux/lib/libcusolver.so.11

In [ ]:


import torch
print('CUDA:',torch.version.cuda)

cudnn = torch.backends.cudnn.version()
cudnn_major = cudnn // 1000
cudnn = cudnn % 1000
cudnn_minor = cudnn // 100
cudnn_patch = cudnn % 100
print( 'cuDNN:', '.'.join([str(cudnn_major),str(cudnn_minor),str(cudnn_patch)]) )



In [ ]:
%%bash
cd models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
# data imports
DATA_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
images_path = os.path.join(DATA_PATH,'train_images')
df_test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")
df_train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
sample_submission = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv")
example = np.load("/kaggle/input/tensorflow-great-barrier-reef/example_test.npy")

In [ ]:
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x)) 

In [ ]:
def bbox_areas(annotations):
    if not annotations:
        return [0]
    area_list = []
    for annotation in annotations:
        area_list.append(annotation['width']*annotation['height'])
    return area_list
df_train["bbox_area"] = df_train["annotations"].apply(bbox_areas)
df_train["max_area"] = df_train["bbox_area"].apply(lambda x : max(x))
df_train["min_area"] = df_train["bbox_area"].apply(lambda x : min(x))



In [ ]:
def img_viz(df_train, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)
df_train.sort_values("max_area", ascending=False).head()

In [ ]:
train_data = df_train.query("Number_bbox>0")
len(train_data)

In [ ]:
train_data = train_data.explode("annotations").reset_index()
train_data = train_data.drop(['index'],axis=1)

In [ ]:
def get_bbox_dim(annotation):
#     print(annotation)
    return [annotation['x'], annotation['y'],annotation['x'] + annotation['width'],annotation['y'] + annotation['height'],  annotation['width'],annotation['height'] ]

In [ ]:
annotations_data = train_data.apply(lambda row: get_bbox_dim(row["annotations"]), axis=1)
annotations_df = pd.DataFrame(list(annotations_data), columns=['xmin', 'ymin', 'xmax', 'ymax', 'width', 'height'])

In [ ]:
from PIL import Image


In [ ]:
train_data = pd.concat([annotations_df, train_data], axis=1)

In [ ]:
xmax = np.array(train_data["xmax"].values.tolist())
ymax = np.array(train_data["ymax"].values.tolist())
train_data["xmax"] = np.where(xmax > 1280, 1280, xmax).tolist()
train_data["ymax"] = np.where(ymax > 720, 720, ymax).tolist()

In [ ]:
train_data['img_path'].iloc[1].split('/')[-1]

In [ ]:
train_data['class'] = 'starfish'
train_data['filename'] = train_data['img_path'].iloc[1].split('/')[-1]

In [ ]:
train_data_video1 = train_data[train_data['video_id']==1].reset_index()

In [ ]:
COLUMNS_TO_KEEP = ['width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'filename','class']
train_csv = train_data_video1[COLUMNS_TO_KEEP]

In [ ]:


from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(train_csv, test_size = 0.2)



In [ ]:
x_train.to_csv('training.csv',index=False)
x_val.to_csv('eval.csv',index=False)


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar --gunzip --extract --verbose --file=faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
open("generate_tfrecord.py","w+")

In [ ]:
%pwd


In [ ]:
%%writefile '/kaggle/working/generate_tfrecord.py'
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.compat.v1.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'wheat':
        return 1
    else:
        return 0


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.compat.v1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.compat.v1.app.run()

In [ ]:
open("label_map.txt","w+")

In [ ]:
%%writefile '/kaggle/working/label_map.txt'
item {
  id: 1
  name: 'starfish'
}

In [ ]:
!pip3 install tf-nightly

In [ ]:
!python /kaggle/working/generate_tfrecord.py --csv_input=/kaggle/working/training.csv  --output_path=train.record --image_dir=../input/tensorflow-great-barrier-reef/train_images/video_0

In [ ]:
!python3 /kaggle/working/generate_tfrecord.py --csv_input=eval.csv  --output_path=eval.record --image_dir=../input/tensorflow-great-barrier-reef/train_images/video_0


In [ ]:


open("pipeline.config","w+")



In [ ]:
%%writefile '/kaggle/working/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config'
model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 5
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  num_steps: 25000
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: .04
          total_steps: 25000
          warmup_learning_rate: .013333
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/kaggle/working/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  data_augmentation_options {
    random_horizontal_flip {
    }
  }

  max_number_of_boxes: 150
  unpad_groundtruth_tensors: false
  use_bfloat16: true  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/kaggle/working/label_map.txt"
  tf_record_input_reader {
    input_path: "/kaggle/working/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/kaggle/working/label_map.txt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/kaggle/working/eval.record"
  }
}

In [ ]:
!mkdir ./tuned

In [ ]:
%cd /kaggle/working/models/research/

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
os.environ['PYTHONPATH'] 

In [ ]:
# PYTHONPATH='/kaggle/working/models'
import os
os.environ['PYTHONPATH'] = "/kaggle/working/models"

import sys
sys.path.append("/kaggle/working/models")
# !export PYTHONPATH=$PYTHONPATH:/kaggle/working/models

In [ ]:
!python object_detection/model_main_tf2.py \
    --num_train_steps=3000 \
    --sample_1_of_n_eval_examples=1 \
    --pipeline_config_path=/kaggle/working/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
    --model_dir=/kaggle/working/tuned \
    --alsologtostderr

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()